In [49]:
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from itertools import permutations
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import io
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

data is the one that I trained on, df is the new test one

In [61]:
data = pd.read_csv(r"D:\Data\wine-reviews\winemag-data-130k-v2.csv")

In [62]:
NUM_WINES = 10

In [63]:
df = pd.read_csv(r"D:\Data\wine-reviews\winemag-data_first150k.csv")

In [64]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [65]:
value_counts = data['variety'].value_counts()
top_wines = list(value_counts[:NUM_WINES].index)

In [66]:
df= df[df.variety.isin(top_wines)]

In [67]:
label_to_idx = {k:v for v, k in enumerate(value_counts.index[:NUM_WINES])}
idx_to_label = {v: k for k, v in label_to_idx.items()}

In [68]:
df["label"] = df['variety'].replace(label_to_idx)
labels = df['label'].values

In [69]:
description_list = df["description"].values
sentences = [sentence + " [SEP] [CLS]" for sentence in description_list]

In [70]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁this', '▁tremendous', '▁100%', '▁', 'vari', 'et', 'al', '▁wine', '▁hail', 's', '▁from', '▁oak', 'ville', '▁and', '▁was', '▁aged', '▁over', '▁three', '▁years', '▁in', '▁oak', '.', '▁juicy', '▁red', '-', 'cher', 'ry', '▁fruit', '▁and', '▁a', '▁compelling', '▁hint', '▁of', '▁caramel', '▁greet', '▁the', '▁palate', ',', '▁framed', '▁by', '▁elegant', ',', '▁fine', '▁', 'tan', 'nin', 's', '▁and', '▁a', '▁subtle', '▁mint', 'y', '▁tone', '▁in', '▁the', '▁background', '.', '▁balanced', '▁and', '▁rewarding', '▁from', '▁start', '▁to', '▁finish', ',', '▁it', '▁has', '▁years', '▁ahead', '▁of', '▁it', '▁to', '▁develop', '▁further', '▁nuance', '.', '▁enjoy', '▁20', '22', '–', '20', '30', '.', '▁[', 's', 'ep', ']', '▁[', 'cl', 's', ']']


In [71]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [72]:
MAX_LEN = 128
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = []

In [73]:
def mask_maker(description):
    z = np.zeros(MAX_LEN)
    position = 0
    count = 1
    for sent in sent_tokenize(description):
        sent_len = len(tokenizer.tokenize(sent))
        z[position:sent_len+1] = count
        position+=sent_len
        count+=1
    
    return z

In [74]:
for d in description_list:
	attention_masks.append(mask_maker(d))

In [75]:
test_inputs = torch.tensor(input_ids.astype(np.int)).long()
test_labels = torch.tensor(labels).long()
test_masks = torch.tensor(attention_masks).long()

In [76]:
labels[90:100]

array([0, 0, 3, 2, 3, 1, 7, 0, 4, 4], dtype=int64)

In [77]:
from collections import Counter

Counter({2: 12800,
         6: 6320,
         0: 14291,
         1: 14482,
         8: 2817,
         7: 5825,
         3: 10062,
         5: 5524,
         4: 7347,
         9: 5070})

In [79]:
model_base = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=10)

In [80]:
model_state_dict = torch.load(save_file)

pytorch_transformers.modeling_xlnet.XLNetForSequenceClassification